# Structured and time series data

This notebook contains an implementation of the 3rd place result in the Rossmann Kaggle competition as detailed in Guo/Berkhahn's [Entity Embeddings of Categorical Variables](https://arxiv.org/abs/1604.06737).

The motivation behind exploring this architecture is it's relevance to real-world application. Most data used for decision making in industry is structured and/or time-series data. Here we explore the end-to-end process of using neural networks with practical structured data problems.

## 0. Preliminaries

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
PATH='data/rossmann/'

import feather    # fast saving/loading of dataframes 
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)
#threshold : int, optional - Total number of array elements which trigger summarization rather than full repr (default 1000).
#edgeitems : int, optional- Number of array items in summary at beginning and end of each dimension (default 3).edgeitems : int, optional

## 1. Create datasets

In addition to the provided data, we will be using external datasets put together by participants in the Kaggle competition. You can download all of them [here](http://files.fast.ai/part2/lesson14/rossmann.tgz).

For completeness, the implementation used to put them together is included below.

In [3]:
def concat_csvs(dirname):
    path = f'{PATH}{dirname}'
    filenames=glob.glob(f"{path}/*.csv")

    wrote_header = False
    with open(f"{path}.csv","w") as outputfile:
        for filename in filenames:
            name = filename.split(".")[0]
            with open(filename) as f:
                line = f.readline()
                if not wrote_header:
                    wrote_header = True
                    outputfile.write("file,"+line)
                for line in f:
                     outputfile.write(name + "," + line)
                outputfile.write("\n")   #Notice this at end of file

NB: Files must finish by "\n", else some of the conversions fail...

### 1.1 Feature Space:
* train: Training set provided by competition
* store: List of stores
* store_states: mapping of store to the German state they are in
* List of German state names
* googletrend: trend of certain google keywords over time, found by users to correlate well w/ given data
* weather: weather
* test: testing set

In [4]:
table_names = ['train', 'store', 'store_states', 'state_names', 
               'googletrend', 'weather', 'test']

`pandas` allows to manipulate tables/data frames in python as one would in a DB.
using `pd.read_csv` to load the above (7) (fname).csv files as dataframes into the list `tables`.

`low_memory` : Internally process the file in chunks, resulting in lower memory use while parsing, but possibly mixed type inference.          

In [5]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False)  
          for fname in table_names]

### 0.1 Missing index in Kaggle table 
test table (v2 from Kaggle) is missing Index which we need for skipping validation set. Add it now. (Or later?)

In [6]:
t = tables[6]
t['Id'] = t.index + 1

In [7]:
from IPython.display import HTML

### 1.2 quick look at the contents of each table:

Use `head()` to get a quick look at the contents of each table:
* train: Contains store information on a daily basis, tracks things like sales, customers, whether that day was a holliday, etc.
* store: general info about the store including competition, etc.
* store_states: maps store to state it is in
* state_names: Maps state abbreviations to names
* googletrend: trend data for particular week/state
* weather: weather conditions for each state
* test: Same as training table, w/o sales and customers


In [8]:
i = 0
for t in tables: 
    print("\nDF:", table_names[i]); i += 1 
    display(t.head(1))


DF: train


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1



DF: store


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN



DF: store_states


...

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96



DF: weather


,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215



DF: test


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0


This is very representative of a typical industry dataset.

The following returns summarized aggregate information to each table accross each field.

### 1.3 summary of each DF using `display(DataFrameSummary(t).summary())`

In [9]:
# for t in tables: display(DataFrameSummary(t).summary())

## 2. Data Cleaning / Feature Engineering

As a structured data problem, we have to go through all the cleaning and feature engineering, even though we're using an ANN.

In [10]:
train, store, store_states, state_names, googletrend, weather, test = tables
print("Training set=", len(train), "Test set =", len(test)) #, len(store), len(store_states), len(state_names), len(googletrend), len(weather)

Training set= 1017209 Test set = 41088


### 2.1 Turn State Holidays to Boolean
We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [11]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

### 2.2 Join Tables with `join_df`

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [12]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Join weather/state names.

In [13]:
weather = join_df(weather, state_names, "file", "StateName")
weather.head(1) 

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW


In pandas you can add new columns to a `dataframe` by simply defining it. We'll do this for googletrends by extracting dates and state names from the given data and adding those columns.

In [14]:
googletrend.head(1)

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96


In [15]:
#Add Date and State columns to this DF
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.head(1)

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN


### 2.3 Review Pandas indexing
We're also going to replace all instances of state name 'NI' to match the usage in the rest of the data: 'HB,NI'. This is a good opportunity to highlight pandas indexing. We can use `.loc[rows, cols]` to select a list of rows and a list of columns from the dataframe. In this case, we're selecting rows w/ statename 'NI' by using a boolean list googletrend.State=='NI' and selecting "State".

In [16]:
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

### 2.4 Using `add_datepart()` to extract particular date fields (from a complete datetime) for constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [17]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

#### The Googletrend data has a special category for the whole of the US - we'll pull that out so we can use it explicitly.

In [18]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

### 2.5 Now outer join all of our data into a single dataframe. 

Recall that in outer joins, everytime a value in the joining field on the left table does not have a corresponding value on the right table, the corresponding row in the new table has Null values for all right table fields. 
One way to check that all records are consistent and complete is to check for Null values post-join, as we do here.

#### *Aside*: Why note just do an inner join?
If you are assuming that all records are complete and match on the field you desire, an inner join will do the same thing as an outer join. However, if you are wrong or a mistake is made, an outer join followed by a null-check will catch it. (Comparing before/after # of rows for inner join is equivalent, but requires keeping track of before/after row #'s. Outer join is easier.)

In [19]:
#Create a DF for the Stores
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [20]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [21]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [22]:
#Use MERGE for the trend_de ... WHY?
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [23]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

#### 2.5 Drop the _y columns in both DFs

In [24]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns: df.drop(c, inplace=True, axis=1)

### 2.6 Fill in missing values to avoid complications with `NA`'s. 

`NA` (Not Available) is how Pandas indicates missing values; Many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.  For example, for Year, the fill for NA is 1900, while for Month and Week we choose 1.

In [25]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

### 2.7 Extract features `CompetitionOpenSince` and `CompetitionDaysOpen`. 
Note the use of `apply()` in mapping a function across dataframe values.

In [26]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

### 2.8 replace erroneous / outlying data 
eg negative days open, etc.

In [27]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

### 2.9 Limit the number of unique categories by adding fields

#### 2.9.1 We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [28]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
#joined.CompetitionMonthsOpen.unique()

#### 2.9.2 Same process for Promo dates.

In [29]:
for df in (joined,joined_test):
    df["Promo2Since"] = pd.to_datetime(df.apply(lambda x: Week(
        x.Promo2SinceYear, x.Promo2SinceWeek).monday(), axis=1).astype(pd.datetime))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

### 2.10 Save data - joined is the training set, joined_tes is the test set

In [30]:
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')
print("Size of training set=", len(joined), "Size of test set=", len(joined_test))

Size of training set= 1017209 Size of test set= 41088


## 3. Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime NA's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [31]:
def get_elapsed(sdf, fld, pre):   # GSG: Added sdf as sorted df parameter
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(sdf.Store.values, sdf[fld].values, sdf.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        ra = ((d-last_date).astype('timedelta64[D]') / day1).astype(int)
        if ra < 0: ra = 0  # GSG: Added check to avoid negative durations
        res.append(ra)       #.astype(int))
        #res.append(((d-last_date).astype('timedelta64[D]') / day1).astype(int))
    sdf[pre+fld] = res
    return sdf

We'll be applying this (get_elapsed) to a subset of columns:

In [32]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [33]:
df_train = train[columns]   #DF for the above train columns to df_train
df_test = test[columns]      #DF for the test set (Jeremy used the same variable, requiring different flow)
#df_train.head(1)

Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [34]:
def add_before_after_count (df1, fld):
    df1 = df1.sort_values(['Store', 'Date'])  #Sort by Store, then Date
    df1 = get_elapsed(df1, fld, 'After')
    df1 = df1.sort_values(['Store', 'Date'], ascending=[True, False])
    df1 = get_elapsed(df1, fld, 'Before')
    return df1

Compute Before and After for SchoolHoliday, StateHoliday, and Promo, for both train and test set.

In [35]:
columnsBaA = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [36]:
for col in columnsBaA:
    df_train = add_before_after_count(df_train, col)
    df_test = add_before_after_count(df_test, col)
#df_train.head(1)

In [37]:
#df_test.head(1)

## 4. Set the active index to Date.

In [38]:
df_train = df_train.set_index("Date")
df_test = df_test.set_index("Date")

#### Then set null values from elapsed field calculations to 0.

In [39]:
for dfx in [df_train, df_test]:
    for o in ['Before', 'After']:
        for p in columnsBaA:
            a = o+p
            dfx[a] = dfx[a].fillna(0)

### 4.1 demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [40]:
#df_train.head(1)
#df_test.head(1)

In [41]:
bwd = df_train[['Store']+columnsBaA].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df_train[['Store']+columnsBaA].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()

In [42]:
for dfx in [bwd, fwd]:
    dfx.drop('Store',1,inplace=True)  #Drop the Store indiced grouped together
    dfx.reset_index(inplace=True)
#fwd.drop('Store',1,inplace=True)
#fwd.reset_index(inplace=True)

In [43]:
df_train.reset_index(inplace=True)
Hdf_train.head(1)

NameError: name 'Hdf_train' is not defined

In [ ]:
df_train = df_train.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df_train = df_train.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])
df_train.drop(columnsBaA,1,inplace=True)
#df_train.head(1)

In [ ]:
bwd = df_test[['Store']+columnsBaA].sort_index().groupby("Store").rolling(7, min_periods=1).sum()
fwd = df_test[['Store']+columnsBaA].sort_index(ascending=False).groupby("Store").rolling(7, min_periods=1).sum()

In [ ]:
### 4.2 Drop the Store indices grouped together in the window function.

In [ ]:
bwd.drop('Store',1,inplace=True)  #Drop the Store indiced grouped together
bwd.reset_index(inplace=True)
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)
df_test.reset_index(inplace=True)

In [ ]:
df_test = df_test.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df_test = df_test.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])
df_test.drop(columnsBaA,1,inplace=True)
df_test.head(1)

In [ ]:
#display(DataFrameSummary(df_train).summary())

### 4.2 drop the Store indices grouped together in the window function.
Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [ ]:
#bwd.drop('Store',1,inplace=True)
#bwd.reset_index(inplace=True)
#bwd.shape
#fwd.drop('Store',1,inplace=True)
#fwd.reset_index(inplace=True)

#for df in [df_train, df_test]:
#    df.reset_index(inplace=True)
#df_train.reset_index(inplace=True)

### 4.3 Now we'll merge these values onto the df.

In [ ]:
#df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
#df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

#df.drop(columnsBaA,1,inplace=True)
#df.head(1)

### 4.4 Back-up large tables
It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [ ]:
df_train.to_feather(f'{PATH}df_train')
df_train = pd.read_feather(f'{PATH}df_train')  #wrong argument? , index_col=0)
df_test.to_feather(f'{PATH}df_test')
df_test = pd.read_feather(f'{PATH}df_test')

In [ ]:
for dfx in [df_train, df_test]:
    dfx["Date"] = pd.to_datetime(dfx.Date)

#print(df_train.columns, "\n\n", df_test.columns)

In [ ]:
joined.shape, df_train.shape,  joined_test.shape, df_test.shape, df.shape

In [ ]:
joined = join_df(joined, df_train, ['Store', 'Date'])
joined_test = join_df(joined_test, df_test, ['Store', 'Date'])

### 4.5 Remove? Instances were the store was closed or had 0 sales.

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [ ]:
#GSG: Testing with avoiding the 0 sales
#joined = joined[joined.Sales!=0]

We'll back this up as well.

In [ ]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)
joined.to_feather(f'{PATH}joined')
joined_test.to_feather(f'{PATH}joined_test')

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## 5. Create features

In [ ]:
joined = pd.read_feather(f'{PATH}joined')
joined_test = pd.read_feather(f'{PATH}joined_test')

Below `T.` is Tranverse

In [ ]:
#joined.head().T.head(20)

## 6. Convert to input compatible for ANN
Now that we've engineered all our features, we need to convert to input compatible with an Artificial Neural Network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc.
If they are floating point, likely to be continous as it will be hard to have enugh categories.
If it is like age, we could decide if to treat them as categorical or continuos.

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

In [ ]:
#joined_test.head().T.head(90)

In [ ]:
#Convert the types of categorical and continuos
for v in cat_vars: 
    joined[v] = joined[v].astype('category').cat.as_ordered()
    # for joined_test it will be done below by "apply_cats"
for v in contin_vars:
    joined[v] = joined[v].astype('float32') #Because pytorch expects all vars as float32
    joined_test[v] = joined_test[v].astype('float32')

In [ ]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

Notice that the test set from Kaggle does not include an 'Id' field.

In [ ]:
#joined.head(1)

In [ ]:
#joined_test.head(1)

In [ ]:
joined_test['Id'] = joined_test.index + 1    #V2 of test was missing the Id field

In [ ]:
joined_test[dep] = 0   # We zero the sales for the test set
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

#joined_test.head(1) 

### 6.1 Use `apply_cats(df, trn)` 
to change any columns of strings in df into categorical variables using trn as
    a template for the category codes.
    This is to guarantee that they have the same set of categories.

In [ ]:
apply_cats(joined_test, joined)

### 6.1 We're going to run on a sample, `joined_samp`

In [ ]:
n = len(joined)
idxs = get_cv_idxs(n, val_pct=150000/n)  #select random set
joined_samp = joined.iloc[idxs].set_index("Date")
samp_size = len(joined_samp); #samp_size

##### 6.1.1 To run on the full dataset, use this instead:

In [ ]:
samp_size = n
joined_samp = joined.set_index("Date")

#print("Sample size=", len(joined_samp), "Shape=", joined_samp.shape)
#joined_samp.head(1)

### We can now process our data...

`proc_df` takes dataframe, pulls variable ('Sales') puts it in y, and returns df without it.
#it also does scaling to 0 to 1. We save `mapper` to do the same to the test set. The mapper saves what is the mean and std deviation of the continuos column. So we later apply the same for the test set.

In [ ]:
df, y, nas, mapper = proc_df(joined_samp, 'Sales', do_scale=True)

In [ ]:
#Avoid 0s returned by proc_df as we will take logs later
y[y==0] = 1

In [ ]:
yl = np.log(y); yl

In [ ]:
df.head(1)  #take a look, now df is all numbers.

In [ ]:
joined_test.head(1)

In [ ]:
joined_test = joined_test.set_index("Date")

In [ ]:
df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

In [ ]:
df_test.head(1)

### NOTE: In time series data, cross-validation is not random. 
Instead, our holdout data is generally the most recent data, as it would be in real application. This issue is discussed in detail in [this post](http://www.fast.ai/2017/11/13/validation-sets/) on our web site.

One approach is to take the last 25% (or 10%) of rows (sorted by date) as our validation set.

In [ ]:
#train_ratio = 0.75
train_ratio = 0.9
train_size = int(samp_size * train_ratio)
val_idx = list(range(train_size, len(df)))
val_idx

An even better option for picking a validation set is using the exact same length of time period as the test set uses - this is implemented below, where
`numpy.flatnonzero(a)` Return indices that are non-zero in the flattened version of a.

In [ ]:
val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))
val_idx

In [ ]:
#val_idx=[0]   #Reset val_idx to use the full training set
#val_idx

## 7. Deep Learning
We're ready to put together our DL models.

### 7.0 Define the metric and related parameters
`Root-Mean-Squared Percent Error` (RMSPE) is the metric Kaggle used for this competition.

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

### 7.1 Create ModelData object
We can create a ModelData object directly from out data frame, passing it the test dataframe (df_test).

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), 
                                       cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

Some categorical variables have a lot more levels than others. Store, in particular, has over a thousand!

In [ ]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]
cat_sz

### 7.2 Embeddings
We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz] #Heuristic for size of embedding
#emb_szs

In [ ]:
#df.head(1)

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars), 0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(100)

### 7.3 Sample

In [ ]:
%time m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars), 0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
%time m.fit(lr, 4, metrics=[exp_rmspe])

In [ ]:
%time m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
%time m.fit(lr, 4, metrics=[exp_rmspe], cycle_len=4)

## 7.4 All

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 1, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

### 7.5 Test

In [ ]:
%time m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
%time m.fit(lr, 4, metrics=[exp_rmspe])

In [ ]:
%time m.fit(lr, 4, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.save('val0')
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
exp_rmspe(x,y)

In [ ]:
pred_test=m.predict(True)  #True to state this is the test set.  predict returns the logs of the predictions

pred_test = np.exp(pred_test)  #now get the actuals
pred_test

In [ ]:
joined_test['Sales']=pred_test

In [ ]:
csv_fn=f'{PATH}tmp/sub.csv'

joined_test[['Id','Sales']].to_csv(csv_fn, index=False)

FileLink(csv_fn)

## 8. RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)

In [ ]:
m = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
m.fit(trn, y_trn);

In [ ]:
preds = m.predict(val)
m.score(trn, y_trn), m.score(val, y_val), m.oob_score_, exp_rmspe(preds, y_val)

In [ ]:
np.exp(preds)
trn
#forest_test = np.
#forest_test['Sales']=np.exp(preds)
#forest_test

In [ ]:
#train.StateHoliday = train.StateHoliday != "0"